<a href="https://colab.research.google.com/github/MaiAbdelhameed/Arabic-Diacritics-Restoration-Project/blob/main/Arabic_Diacitrics_Restoration_last_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

for PyArabic
https://github.com/linuxscout/pyarabic/blob/master/doc/features.md

In [ ]:
!pip install pyarabic
import nltk
nltk.download('punkt')
import re
import pyarabic.araby as araby
from pyarabic.araby import is_arabicrange, strip_diacritics, sentence_tokenize
import gensim
from gensim.models import Word2Vec
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy
from numpy import array
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Inputs


In [ ]:
 def read_input(file_path):
    with open(file_path, 'r') as file:
        content =  file.readlines()
    return content

In [ ]:
train_dataset = read_input('train.txt')
val_dataset = read_input('val.txt')

#Preprocessing

In [ ]:
def remove_diacritics(text):
    diacritics = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
    stripped = araby.strip_diacritics(text)
    diac = []
    diac_indx=[]
    for i, char in enumerate(text):
        if char not in stripped:
            diac.append(char)
            diac_indx.append(i)
        else:
            if (i <len(text) -1 and text[i+1] not in diacritics):
                diac.append('*')
                diac_indx.append(i)
            elif i == len(text) -1:
                diac.append('*')
                diac_indx.append(i+1)

    return stripped, diac, diac_indx

In [ ]:
tst = "مَّرحّ"
stripped, dia, diac_ind = remove_diacritics(tst)
print(dia)
print(diac_ind)

['ّ', 'َ', '*', 'ّ']
[1, 2, 3, 5]


In [ ]:
"َّ" == "َّ"

False

In [ ]:
def remove_non_arabic_char(text):
    PUNCTUATIONS = [".", "،", ":", "؛", "-", "؟"]
    regex = re.compile('[^؀-ۿ ]')
    result = regex.sub('', text)
    arabic_punctuation_pattern = re.compile('[-|؟|!|,|.\|:|؛|\"|\'،]+')
    result_text = arabic_punctuation_pattern.sub('', result)
    return result_text

In [ ]:
tst = "كم عمرك يا علي"
remove_non_arabic_char(tst)

'كم عمرك يا علي'

In [ ]:
def segmentation(text):
    segmented = sentence_tokenize(text)
    if (len(segmented[0]) > 10): # if length of segmented sentence is too long, chop into segements of 10 (or less)
        words_per_segment = 10
        words = segmented[0].split()
        segmented_sentences = []

        for i in range(0, len(words), words_per_segment):
            segment = ' '.join(words[i:i+words_per_segment])
            segmented_sentences.append(segment)
        return segmented_sentences
    else:
        return  segmented

# Dataset

In [ ]:
class ArabicTextDataset:
    def __init__(self, dataset):
        self.dataset = dataset
        self.original_text = []
        self.without_non_arabic = []
        self.without_diacritics = []
        self.diacritics = []
        self.diacritics_indices = []

    def preprocess_text(self, text):
        arabic_only = remove_non_arabic_char(text)
        stripped, dia, dia_indx = remove_diacritics(arabic_only)
        return arabic_only, stripped, dia, dia_indx

    def process_dataset(self):
        for t in self.dataset:
            segmented = segmentation(t)
            for s in segmented:
                without_non_arabic, without_diacritics, diacritics, dia_indices = self.preprocess_text(s)
                self.original_text.append(s)
                self.without_non_arabic.append(without_non_arabic)
                self.without_diacritics.append(without_diacritics)
                self.diacritics.append(diacritics)
                self.diacritics_indices.append(dia_indices)

    def create_dataframe(self):
        data = {
            'original_text': self.original_text,
            'arabic_only': self.without_non_arabic,
            'without_diacritics': self.without_diacritics,
            'diacritics' : self.diacritics,
            'diacritics_indices': self.diacritics_indices
        }
        return pd.DataFrame(data)

In [ ]:
dataset_processor = ArabicTextDataset(train_dataset)
dataset_processor.process_dataset()
train_df = dataset_processor.create_dataframe()

In [ ]:
train_df

,original_text,arabic_only,without_diacritics,diacritics,diacritics_indices
0,قَوْلُهُ :,قَوْلُهُ,قوله,"[َ, ْ, ُ, ُ, *]","[1, 3, 5, 7, 9]"
1,( أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ ) قَالَ ...,أَوْ قَطَعَ الْأَوَّلُ يَدَهُ إلَخْ قَالَ ال...,أو قطع الأول يده إلخ قال الزركشي,"[*, َ, ْ, *, َ, َ, َ, *, *, ْ, َ, ّ, َ, ُ, *, ...","[0, 2, 4, 5, 7, 9, 11, 12, 13, 15, 17, 19, 20,..."
2,ابْنُ عَرَفَةَ :,ابْنُ عَرَفَةَ,ابن عرفة,"[*, ْ, ُ, *, َ, َ, َ, َ, *]","[0, 2, 4, 5, 7, 9, 11, 13, 15]"
3,( قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ ) أَ...,قَوْلُهُ لِعَدَمِ مَا تَتَعَلَّقُ إلَخْ أَيْ...,قوله لعدم ما تتعلق إلخ أي الوصية,"[*, َ, ْ, ُ, ُ, *, ِ, َ, َ, ِ, *, َ, *, *, َ, ...","[0, 2, 4, 6, 8, 9, 11, 13, 15, 17, 18, 20, 21,..."
4,قَوْلُهُ مَا مَرَّ ) أَيْ قُبَيْلَ قَوْلِ الْم...,قَوْلُهُ مَا مَرَّ أَيْ قُبَيْلَ قَوْلِ الْمَ...,قوله ما مر أي قبيل قول المتن لغت ولو,"[َ, ْ, ُ, ُ, *, َ, *, *, َ, ّ, َ, *, *, َ, ْ, ...","[1, 3, 5, 7, 8, 10, 11, 12, 14, 16, 17, 18, 19..."
...,...,...,...,...,...
136591,لِأَمْرٍ بِمَعْنَى أَنَّهُ يُؤَمَّنُ لِنُزُولِ...,لِأَمْرٍ بِمَعْنَى أَنَّهُ يُؤَمَّنُ لِنُزُولِ...,لأمر بمعنى أنه يؤمن لنزوله لأرض الإسلام لشراء ...,"[ِ, َ, ْ, ٍ, *, ِ, َ, ْ, َ, *, *, َ, ّ, َ, ُ, ...","[1, 3, 5, 7, 8, 10, 12, 14, 16, 17, 18, 20, 22..."
136592,فَرَغَ سَبَبُهُ انْصَرَفَ الْأَمَانُ وَهَذَا ا...,فَرَغَ سَبَبُهُ انْصَرَفَ الْأَمَانُ وَهَذَا ا...,فرغ سببه انصرف الأمان وهذا القيد أخرج به المها...,"[َ, َ, َ, *, َ, َ, ُ, ُ, *, *, ْ, َ, َ, َ, *, ...","[1, 3, 5, 6, 8, 10, 12, 14, 15, 16, 18, 20, 22..."
136593,كَمَا ذَكَرَهُ فِي ك وَلَا يَخْفَى أَنَّهُ لَا...,كَمَا ذَكَرَهُ فِي ك وَلَا يَخْفَى أَنَّهُ لَا...,كما ذكره في ك ولا يخفى أنه لا يشمل صور,"[َ, َ, *, *, َ, َ, َ, ُ, *, ِ, *, *, *, *, َ, ...","[1, 3, 4, 5, 7, 9, 11, 13, 14, 16, 17, 18, 19,..."
136594,الِاسْتِئْمَانِ كُلَّهَا فَإِنَّهُ لَا يَشْمَل...,الِاسْتِئْمَانِ كُلَّهَا فَإِنَّهُ لَا يَشْمَل...,الاستئمان كلها فإنه لا يشمل ما إذا دخل على الإ...,"[*, ِ, *, ْ, ِ, ْ, َ, *, ِ, *, ُ, ّ, َ, َ, *, ...","[0, 2, 3, 5, 7, 9, 11, 12, 14, 15, 17, 19, 20,..."


In [ ]:
csv_filename = 'train_dataframe.csv'
train_df.to_csv(csv_filename, index=True)

#Feature Extraction

In [ ]:
diacritics = ["ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ"]
#* represents empty string => no tashkeel
possible_diacritics = ["*", "ْ", "ّ", "ٌ", "ٍ", "ِ", "ً", "َ", "ُ", "َّ", "ًّ", "ُّ", "ٌّ", "ِّ", "ٍّ"]
arab_chars = "ىعظحرسيشضق ثلصطكآماإهزءأفؤغجئدةخوبذتن"
punctuation = [".", "،", ":", "؛", "-", "؟", "!"]
arab_chars_list = [char for char in arab_chars]

In [ ]:
encoder_alphabet = OneHotEncoder(sparse=False, categories=[arab_chars_list])
encoder_diacritics = OneHotEncoder(sparse = False, categories = [possible_diacritics])

one_hot_encoded_alphabet = encoder_alphabet.fit_transform([[category] for category in arab_chars_list])
one_hot_encoded_diacritics = encoder_diacritics.fit_transform([[category] for category in possible_diacritics])

print(one_hot_encoded_alphabet.shape)
print(one_hot_encoded_diacritics.shape)

(37, 37)
(15, 15)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
def apply_one_hot_encoding(char_list, encoder):
    one_hot_encoded = encoder.transform([[char] for char in char_list])
    return one_hot_encoded.tolist()

In [ ]:
words = []
words_diacritics = []

sentences = train_df['arabic_only']
for sentence in sentences:
    temp_words = []
    temp_diacritics = []

    for word in sentence.split():
        stripped, dia, dia_indices = remove_diacritics(word)
        temp_words.append(stripped)
        temp_diacritics.append(dia)

    words.extend(temp_words)
    words_diacritics.extend(temp_diacritics)

In [ ]:
indices = [i for i, lst in enumerate(words_diacritics) if len(lst) == 0]

In [ ]:
len(indices)

0

In [ ]:
len(words_diacritics)

920431

In [ ]:
for ind in indices:
    print(f"{words_diacritics[ind]} to word = {words[ind]}")

In [ ]:
# list_of_lists = [[], [1, 2, 3], [], [4, 5], []]

# updated_list_of_lists = [['*' if not inner_list else element for element in inner_list] if inner_list else ['*'] for inner_list in list_of_lists]

# print(updated_list_of_lists)

In [ ]:
indices = [i for i, lst in enumerate(words_diacritics) if len(lst) == 0]
len(indices)

0

In [ ]:
# harakka = shadda + "َ"
# len(harakka)

In [ ]:
words_diacritics

[['َ', 'ْ', 'ُ', 'ُ'],
 ['َ', 'ْ'],
 ['َ', 'َ', 'َ'],
 ['*', 'ْ', 'َ', 'ّ', 'َ', 'ُ'],
 ['َ', 'َ', 'ُ'],
 ['*', 'َ', 'ْ'],
 ['َ', '*', 'َ'],
 ['*', '*', 'ّ', 'َ', 'ْ', 'َ', 'ِ', 'ّ', 'ُ'],
 ['*', 'ْ', 'ُ'],
 ['َ', 'َ', 'َ', 'َ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['ِ', 'َ', 'َ', 'ِ'],
 ['َ', '*'],
 ['َ', 'َ', 'َ', 'ّ', 'َ', 'ُ'],
 ['*', 'َ', 'ْ'],
 ['َ', 'ْ'],
 ['*', 'ْ', 'َ', 'ِ', 'ّ', 'َ', 'ُ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['َ', '*'],
 ['َ', 'ّ', 'َ'],
 ['َ', 'ْ'],
 ['ُ', 'َ', 'ْ', 'َ'],
 ['َ', 'ْ', 'ِ'],
 ['*', 'ْ', 'َ', 'ْ', 'ِ'],
 ['َ', 'َ', 'ْ'],
 ['َ', 'َ', 'ْ'],
 ['*', 'ْ', 'َ', 'َ', 'َ'],
 ['َ', 'َ', '*'],
 ['َ', 'ْ', 'َ', 'ْ', '*'],
 ['َ', 'ُ'],
 ['ِ', 'َ', '*', 'ٍ'],
 ['َ', 'ْ'],
 ['َ', 'ْ', 'ُ', '*', 'ُ'],
 ['َ', '*', 'ً'],
 ['َ', 'َ', '*'],
 ['َ', 'َ', 'َ'],
 ['َ', 'ُ'],
 ['ِ', 'ْ', 'َ'],
 ['*', 'ْ', 'َ', 'ْ', 'ِ'],
 ['َ', 'ْ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['*', 'ْ', 'َ', 'ِ', 'ّ', 'َ', 'ُ'],
 ['َ', 'ْ'],
 ['ُ', 'ْ', 'َ', 'َ', '*'],
 ['َ', 'ُ'],
 ['َ', '*', 'ٌ'],
 ['َ', 'ُ', 'ْ', 'َ', 'ُ'],

In [ ]:
new_words_diacritics = []
shadda = "ّ"
fatha = "َ"
kasra = "ِ"
damma = "ُ"
fatheten = "ً"
dammatan = "ٌ"
kasratan = "ٍ"
available_shadda_tashkeel = [fatha] + [kasra] + [damma] + [fatheten] + [dammatan]+[kasratan]
num_shaddas = 0
for ind, dia in enumerate(words_diacritics):
    print(ind)
    i=0
    temp = []
    # print(dia)
    while i < len(dia) and len(dia) != 0:
        if dia[i] == shadda and i < len(dia) - 1:
            if dia[i+1] in available_shadda_tashkeel:
                merged_element = dia[i] + dia[i + 1]
                temp.append(merged_element)
                i += 2  # Skip the next element since it has been merged
                num_shaddas +=1
            else:
                temp.append(dia[i])
                i += 1

        else:
            temp.append(dia[i])
            i += 1
    new_words_diacritics.append(temp)

Streaming output truncated to the last 5000 lines.
915431
915432
915433
915434
915435
915436
915437
915438
915439
915440
915441
915442
915443
915444
915445
915446
915447
915448
915449
915450
915451
915452
915453
915454
915455
915456
915457
915458
915459
915460
915461
915462
915463
915464
915465
915466
915467
915468
915469
915470
915471
915472
915473
915474
915475
915476
915477
915478
915479
915480
915481
915482
915483
915484
915485
915486
915487
915488
915489
915490
915491
915492
915493
915494
915495
915496
915497
915498
915499
915500
915501
915502
915503
915504
915505
915506
915507
915508
915509
915510
915511
915512
915513
915514
915515
915516
915517
915518
915519
915520
915521
915522
915523
915524
915525
915526
915527
915528
915529
915530
915531
915532
915533
915534
915535
915536
915537
915538
915539
915540
915541
915542
915543
915544
915545
915546
915547
915548
915549
915550
915551
915552
915553
915554
915555
915556
915557
915558
915559
915560
915561
915562
915563
915564
915565
9155

In [ ]:
len(new_words_diacritics)

920431

In [ ]:
new_words_diacritics

[['َ', 'ْ', 'ُ', 'ُ'],
 ['َ', 'ْ'],
 ['َ', 'َ', 'َ'],
 ['*', 'ْ', 'َ', 'َّ', 'ُ'],
 ['َ', 'َ', 'ُ'],
 ['*', 'َ', 'ْ'],
 ['َ', '*', 'َ'],
 ['*', '*', 'َّ', 'ْ', 'َ', 'ِ', 'ُّ'],
 ['*', 'ْ', 'ُ'],
 ['َ', 'َ', 'َ', 'َ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['ِ', 'َ', 'َ', 'ِ'],
 ['َ', '*'],
 ['َ', 'َ', 'َ', 'َّ', 'ُ'],
 ['*', 'َ', 'ْ'],
 ['َ', 'ْ'],
 ['*', 'ْ', 'َ', 'ِ', 'َّ', 'ُ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['َ', '*'],
 ['َ', 'َّ'],
 ['َ', 'ْ'],
 ['ُ', 'َ', 'ْ', 'َ'],
 ['َ', 'ْ', 'ِ'],
 ['*', 'ْ', 'َ', 'ْ', 'ِ'],
 ['َ', 'َ', 'ْ'],
 ['َ', 'َ', 'ْ'],
 ['*', 'ْ', 'َ', 'َ', 'َ'],
 ['َ', 'َ', '*'],
 ['َ', 'ْ', 'َ', 'ْ', '*'],
 ['َ', 'ُ'],
 ['ِ', 'َ', '*', 'ٍ'],
 ['َ', 'ْ'],
 ['َ', 'ْ', 'ُ', '*', 'ُ'],
 ['َ', '*', 'ً'],
 ['َ', 'َ', '*'],
 ['َ', 'َ', 'َ'],
 ['َ', 'ُ'],
 ['ِ', 'ْ', 'َ'],
 ['*', 'ْ', 'َ', 'ْ', 'ِ'],
 ['َ', 'ْ'],
 ['َ', 'ْ', 'ُ', 'ُ'],
 ['*', 'ْ', 'َ', 'ِ', 'َّ', 'ُ'],
 ['َ', 'ْ'],
 ['ُ', 'ْ', 'َ', 'َ', '*'],
 ['َ', 'ُ'],
 ['َ', '*', 'ٌ'],
 ['َ', 'ُ', 'ْ', 'َ', 'ُ'],
 ['ِ', 'ْ'],
 ['َ', 'ْ', 'ِ

In [ ]:
chars_of_words = [list(word) for word in words]
print(chars_of_words[:5])
print(len(words))
print(len(chars_of_words))
print(len(new_words_diacritics))

[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل'], ['ي', 'د', 'ه']]
920431
920431
920431


In [ ]:
# encoder_alphabet, encoder_diacritics
word_encoded = apply_one_hot_encoding(chars_of_words[0], encoder_alphabet)
diac_encoded = apply_one_hot_encoding(new_words_diacritics[0], encoder_diacritics)
print(word_encoded)
print(diac_encoded)
print(len(word_encoded))
print(len(diac_encoded))

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0

In [ ]:
# categories = ["cat", "dog", "bird", ""]  # "" represents the empty string

# # Create a one-hot encoding dictionary
# one_hot_encoding = {category: [1 if category == cat else 0 for cat in categories] for category in categories}

# # Apply one-hot encoding
# empty_string_encoded = one_hot_encoding[""]
# print(empty_string_encoded)

In [ ]:
words_diac_df = pd.DataFrame()

In [ ]:
words_encoding = []
for chars in chars_of_words:
    word_encoded = apply_one_hot_encoding(chars, encoder_alphabet)
    words_encoding.append(word_encoded)

In [ ]:
words_diac_df['words_encoding'] = words_encoding

In [ ]:
diac_encoding = []
for dia in new_words_diacritics:
    diac_encoded = apply_one_hot_encoding(dia, encoder_diacritics)
    diac_encoding.append(diac_encoded)

In [ ]:
words_diac_df['diac_encoding'] = diac_encoding

In [ ]:
csv_filename = 'word_diac_encoding_dataframe.csv'
words_diac_df.to_csv(csv_filename, index=True)

In [ ]:
max_word = 0
for i in words_encoding:
    if len(i) > max_word:
        max_word = len(i)

max_word

12

TypeError: ignored

# RNN for feature extraction

In [ ]:
word_diac_encoding_df = pd.read_csv("/content/word_diac_encoding_dataframe.csv")

In [ ]:
word_diac_encoding_df

,Unnamed: 0,words_encoding,diac_encoding
0,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
1,1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
2,2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
3,3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
...,...,...,...
920426,920426,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
920427,920427,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
920428,920428,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,..."
920429,920429,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."


In [ ]:
words_encoding = word_diac_encoding_df['words_encoding']
diac_encoding = word_diac_encoding_df['diac_encoding']
max_word = 0
word_index = 0
for index, ind in enumerate(words_encoding):
    if len(ind) > max_word:
        max_word = len(ind)
        word_index = index
max_word


2244

In [ ]:
# Find the length of the longest sublist
max_length = max_word

# Pad shorter sublists with zeros to make them of equal length
padded_list = [sublist + [0] * (max_length - len(sublist)) for sublist in words_encoding]
print(padded_list)
# Convert the padded 2D list to a TensorFlow tensor
my_tensor = tf.constant(padded_list)

TypeError: ignored

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sample dataset
X_one_hot = tf.constant(words_encoding)
y_one_hot = tf.constant(diac_encoding)
num_word_classes = len(arab_chars)
embedding_dim = 300
lstm_units = 300
max_sequence_length = max_word
num_diac_classes = len(possible_diacritics)
num_epochs = 20
batch_size = 100



model = Sequential()
# Input layer: Embedding layer to convert one-hot encoded words to dense vectors
model.add(Embedding(input_dim=num_word_classes, output_dim=embedding_dim, input_length=max_sequence_length))

# LSTM layer to process the sequence of word embeddings
model.add(LSTM(units=lstm_units, return_sequences=True))

# Dense layer with softmax activation for diacritics prediction at each time step
model.add(TimeDistributed(Dense(units=num_diac_classes, activation='softmax')))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model using teacher forcing
model.fit(X_one_hot, y_one_hot, epochs=num_epochs, batch_size=batch_size)

ValueError: ignored

In [ ]:
# Create Word2Vec model (using Skip-gram, you can also use CBOW)
model = Word2Vec(sentences=sentences, vector_size=300, window=20, sg=1, min_count=1)
model.train(sentences, total_examples=len(sentences), epochs=10)
# Save the model (optional)
model.save("word2vec_model")

# Load the model (optional)
# model = Word2Vec.load("word2vec_model")

word = "أو"
# Get the vector representation of a word
word_vector = model.wv[word]

# Find similar words
similar_words = model.wv.most_similar(word, topn=5)

# Print the results
print(f"Vector representation of {word}:", word_vector)
print(f"Similar words to {word}:", similar_words)

Vector representation of أو: [-6.99151084e-02  3.05549949e-01 -1.71290725e-01 -1.68057159e-02
  1.93623349e-01 -2.39820838e-01  2.10489348e-01  4.52250868e-01
  1.24369346e-01 -6.09374642e-02 -2.86353249e-02 -4.67340433e-04
  2.16338217e-01  3.36695672e-03 -5.73583357e-02 -9.78622679e-03
  1.45897582e-01  2.13593945e-01 -2.97793075e-02  1.07618831e-01
 -2.45435476e-01  3.40700485e-02 -4.71212715e-02 -1.23616494e-01
  1.47061974e-01 -1.41845271e-01 -2.65673310e-01  6.04879782e-02
 -1.70172029e-03 -2.09539101e-01 -2.46850178e-01 -3.65248993e-02
  2.08183825e-01 -1.12353303e-01 -6.57649264e-02  1.42010152e-01
 -1.03371799e-01 -1.09847613e-01 -4.94952016e-02 -1.02198392e-01
 -7.38092586e-02 -1.11796044e-01 -1.54698536e-01  1.62741728e-02
 -3.30022983e-02  1.01303622e-01 -3.92164886e-02  1.62024185e-01
  2.93581307e-01  1.92659333e-01  1.52386665e-01  2.95715183e-02
  6.80400282e-02 -8.55665058e-02 -2.23495029e-02  2.51394585e-02
  4.59075458e-02  3.02601587e-02 -9.24976449e-03 -2.09428534e

In [ ]:
word_vector = model.wv["قال"]

word = "جميل"
# Find similar words
similar_words = model.wv.most_similar(word, topn=5)

# Print the results
print(f"Vector representation of {word}:", word_vector)
print(f"Similar words to {word}:", similar_words)

AttributeError: ignored